In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/lvtetmesh_coarse_fibers_LDRB_1.vtk") 
lv = LV.from_pyvista_read(filepath)

In [3]:
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.APEX_BASE_REGIONS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.EPI_ENDO, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.SURFS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.AM_SURFS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.SURFS_DETAILED, overwrite=False)

In [4]:
s = lv.create_surface_oi_from_surface(LV_MESH_DATA.SURFS_DETAILED)

In [5]:
lv.set_aortic_info()
lv.set_mitral_info()

In [6]:
aortic_bc = lv.create_spring_rim_bc("AORTIC_BC", LV_SURFS.BORDER_AORTIC)
mitral_bc = lv.create_spring_rim_bc("MITRAL_BC", LV_SURFS.BORDER_MITRAL)

In [7]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Boundary: 1
-> Loads: 1
-> Discrete: 2
-> LoadData: 3
-> Output: 1
-> MeshData: 0

In [8]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template_simple.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])
feb.write("../sample_files/sample_lv_template_simple_mod1.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 2
--> Nodes 'LV': 64452
--> Elements 'LV': 218359
-> Boundary: 1
-> LoadData: 3
-> Output: 1

In [9]:
N_NODES = lv.mesh.n_points

In [10]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template_simple.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])

feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + 1)

feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])
feb.write("../sample_files/sample_lv_template_simple_mod2.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 3
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Elements 'LV': 218359
-> Boundary: 1
-> LoadData: 3
-> Output: 1

In [11]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template_simple.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])

feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + 1)

feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])

feb.add_elements([
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1
    }
  ])

feb.write("../sample_files/sample_lv_template_simple_mod3.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 4
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
-> Boundary: 1
-> LoadData: 3
-> Output: 1

In [12]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template_simple.feb")
feb = FEBio_feb.from_file(feb_template_path)

# ----------------------
# Add Nodes
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])

feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + 1)


# ----------------------
# Add Elements
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])

feb.add_elements([
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1
    }
  ])

feb.write("../sample_files/sample_lv_template_simple_mod3.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 4
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
-> Boundary: 1
-> LoadData: 3
-> Output: 1

In [13]:
adjusted_discrete_sets = {}
to_adj = N_NODES
for key, values in lv._discrete_sets.items():
  adj_vals = np.copy(values) + 1
  adj_vals[:, 1] += to_adj
  adjusted_discrete_sets[key] = adj_vals
  to_adj += len(lv.get_bc(key)[1]["RIM_NODES"]) # account for nodes

In [14]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template_simple_2.feb")
feb = FEBio_feb.from_file(feb_template_path)

# ----------------------
# Add Nodes
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])

feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + 1)


# ----------------------
# Add Elements
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])

feb.add_elements([
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1
    }
  ])


# ----------------------
# Add Discrete set
import xml.etree.ElementTree as ET
feb.add_discretesets({"AORTIC_RIM_ELEMS": adjusted_discrete_sets["AORTIC_BC"]})
discrete = feb.discrete()
discrete.find("discrete").attrib["discrete_set"] = "AORTIC_RIM_ELEMS"


feb.write("../sample_files/sample_lv_template_simple_mod4.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 5
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
-> Boundary: 1
-> Discrete: 2
-> LoadData: 3
-> Output: 1

In [15]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template_simple_2.feb")
feb = FEBio_feb.from_file(feb_template_path)

# ----------------------
# Add Nodes
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])

feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + 1)

N_NODES_AORTIC = len(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"])

feb.add_nodes([
  {"name": "MITRAL_RIM", 
   "nodes": np.round(lv.get_bc("MITRAL_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + N_NODES_AORTIC + 1)


# ----------------------
# Add Elements
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])

feb.add_elements([
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1
    }
  ])

N_ELEMS_AORTIC = len(aortic_bc[LV_RIM.ELEMENTS.value])

feb.add_elements([
  {
    "name": "MITRAL_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": mitral_bc[LV_RIM.ELEMENTS.value] + N_NODES + N_NODES_AORTIC + 1
    }
  ], 
  initial_el_id = N_ELEMS_AORTIC + 1)


# ----------------------
# Add Discrete set
feb.add_discretesets({"AORTIC_RIM_ELEMS": adjusted_discrete_sets["AORTIC_BC"]})
feb.add_discretesets({"MITRAL_RIM_ELEMS": adjusted_discrete_sets["MITRAL_BC"]})

discrete = feb.discrete()
discrete.find("discrete").attrib["discrete_set"] = "AORTIC_RIM_ELEMS"

import xml.etree.ElementTree as ET
subel = ET.SubElement(discrete, "discrete")
subel.set("discrete_set","MITRAL_RIM_ELEMS")
subel.set("dmat","1")

feb.write("../sample_files/sample_lv_template_simple_mod6.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 8
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'MITRAL_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
--> Elements 'MITRAL_RIM_ELEMS': 63
-> Boundary: 1
-> Discrete: 3
-> LoadData: 3
-> Output: 1

In [16]:
from febio_python.feb import FEBio_feb
feb_template_path = Path("../sample_files/sample_lv_template.feb")
feb = FEBio_feb.from_file(feb_template_path)

# ----------------------
# Add Nodes
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  ])

feb.add_nodes([
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + 1)

N_NODES_AORTIC = len(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"])

feb.add_nodes([
  {"name": "MITRAL_RIM", 
   "nodes": np.round(lv.get_bc("MITRAL_BC")[1]["RIM_NODES"], 4)
   },
  ],
  initial_el_id= N_NODES + N_NODES_AORTIC + 1)


# ----------------------
# Add Elements
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    }
  ])

feb.add_elements([
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1
    }
  ])

N_ELEMS_AORTIC = len(aortic_bc[LV_RIM.ELEMENTS.value])

feb.add_elements([
  {
    "name": "MITRAL_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": mitral_bc[LV_RIM.ELEMENTS.value] + N_NODES + N_NODES_AORTIC + 1
    }
  ], 
  initial_el_id = N_ELEMS_AORTIC + 1)

# ----------------------
# Add Surfaces
feb.add_surfaces({"ENDO": np.vstack(lv.get_surface_oi(LV_SURFS.ENDO)) + 1})

# ----------------------
# Add Discrete set
feb.add_discretesets({"AORTIC_RIM_ELEMS": adjusted_discrete_sets["AORTIC_BC"]})
feb.add_discretesets({"MITRAL_RIM_ELEMS": adjusted_discrete_sets["MITRAL_BC"]})

discrete = feb.discrete()
discrete.find("discrete").attrib["discrete_set"] = "AORTIC_RIM_ELEMS"

import xml.etree.ElementTree as ET
subel = ET.SubElement(discrete, "discrete")
subel.set("discrete_set","MITRAL_RIM_ELEMS")
subel.set("dmat","1")

# ----------------------
# Add fibers
feb.add_meshdata([
  {
    "elem_set": "LV", 
    "var": "mat_axis",
    "elems": {
      "a": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.F0),
      "d": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.S0),
      }
    }
  ])

feb.write("../sample_files/sample_lv_template_mod.feb")
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 9
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'MITRAL_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
--> Elements 'MITRAL_RIM_ELEMS': 63
-> Boundary: 1
-> Loads: 1
-> Discrete: 3
-> LoadData: 3
-> Output: 1
-> MeshData: 1